## XGBoost multi instrument

This code is an attempt to make a classification of multiple instruments base on a single, combined spectrogram using XGBoost

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import external functions from the functions folder
import sys
sys.path.append('../../functions')
import functions as f


In [2]:
# Set local path to the folder containing the .wav audio files
path = 'C:/Users/lucvo/VScode/Machine_learning/Audio_data/nsynth-valid.jsonwav/nsynth-valid/audio/'

In [3]:
# Generate the dataset
X, y = f.generate_mixed_spectrograms(1000, 3, path=path)

# Flatten the dataset
X = X.reshape(X.shape[0], -1)

# Split into training, validation and test (80/10/10)
X_train, X_val, X_test, y_train, y_val, y_test = f.split_data(X, y, 0.1, 0.1)

100%|██████████| 1000/1000 [00:08<00:00, 121.79it/s]


In [4]:
# Import XGBoost
import xgboost as xgb
import tqdm as tqdm

In [5]:
# Train the multi-class classification model
models = []
results = []
N_intstruments = 6

for i in tqdm.tqdm(range(N_intstruments)):
    y_train_cut = y_train[:, i]
    y_val_cut = y_val[:, i]

    model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, max_depth=3, learning_rate=0.1, n_jobs=-1)
    eval_set = [(X_train, y_train_cut), (X_val, y_val_cut)]
    model.fit(X_train, y_train_cut, eval_set=eval_set, early_stopping_rounds=10, verbose=True)
    models.append(model)
    results.append(model.evals_result())

# Evaluate the model
y_pred = np.zeros(y_test.shape)
for i in range(N_intstruments):
    y_pred[:, i] = models[i].predict(X_test)

# Calculate the accuracy
accuracy = np.mean(y_pred == y_test)
print(f'Accuracy: {accuracy}')

  0%|          | 0/6 [00:00<?, ?it/s]c:\Users\lucvo\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.65744	validation_1-logloss:0.66961
[1]	validation_0-logloss:0.62756	validation_1-logloss:0.64940
[2]	validation_0-logloss:0.59728	validation_1-logloss:0.63752
[3]	validation_0-logloss:0.57287	validation_1-logloss:0.62486
[4]	validation_0-logloss:0.54919	validation_1-logloss:0.61177
[5]	validation_0-logloss:0.52818	validation_1-logloss:0.60745
[6]	validation_0-logloss:0.50923	validation_1-logloss:0.59515
[7]	validation_0-logloss:0.48963	validation_1-logloss:0.58771
[8]	validation_0-logloss:0.47099	validation_1-logloss:0.58259
[9]	validation_0-logloss:0.45610	validation_1-logloss:0.57456
[10]	validation_0-logloss:0.44139	validation_1-logloss:0.56777
[11]	validation_0-logloss:0.42805	validation_1-logloss:0.56101
[12]	validation_0-logloss:0.41354	validation_1-logloss:0.55580
[13]	validation_0-logloss:0.39941	validation_1-logloss:0.54387
[14]	validation_0-logloss:0.38785	validation_1-logloss:0.53900
[15]	validation_0-logloss:0.37571	validation_1-logloss:0.53560
[1

  0%|          | 0/6 [05:32<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Print the accuracy for each instrument
instruments = ['organ', 'bass', 'guitar', 'vocal', 'flutes', 'keyboards']
print('Accuracies for 1600 training samples:')
for i in range(N_intstruments):
    accuracy = np.mean(y_pred[:, i] == y_test[:, i])
    print(f'Accuracy for instrument {instruments[i]}: {accuracy}')

# Total accuracy
accuracy = np.mean(y_pred == y_test)
print()
print(f'Total accuracy: {accuracy}')

Accuracies for 1600 training samples:
Accuracy for instrument organ: 0.74
Accuracy for instrument bass: 0.62
Accuracy for instrument guitar: 0.67
Accuracy for instrument vocal: 0.96
Accuracy for instrument flutes: 0.84
Accuracy for instrument keyboards: 0.65

Total accuracy: 0.7466666666666667


In [ ]:
# Save the models
import pickle
for i in range(N_intstruments):
    instruments = ['organ', 'bass', 'guitar', 'vocal', 'flutes', 'keyboards']
    filenames = [f'XGBoost_multi_inst_model_{instrument}.pkl' for instrument in instruments]
    with open(filenames[i], 'wb') as file:
        pickle.dump(models[i], file)